# task_One (Python Parser)
_Date : 03/08/2017_

## Goals for task_One
1. Create a parser to build insectGenomesInfos.csv
2. Update Leo pipeline Gene\_Prediction\_Genome\_DB by used __Evidence Base Modeler__ for finding ORF

### Final Goal 
__Show the phylogenetic distribution of oskar in all insect orders__
> - Could be very similar to this figure, which contains information about (1) how many species’ genomes/transcriptomes were searched in each insect Order, and (2) whether oskar was found for each species.
> - Could simplify, and indicate whether oskar was found at least once in that order – ie, instead of having many data points per order, have only one Y/N point per order.


## Done 
- Add parser options in Python parser
- Add checkIfInsect() function in Python parser



In [13]:
'''
Created date : 03-08-2017
Parser part 
'''

import pandas as pd
import os
from optparse import OptionParser


parser = OptionParser()
parser.add_option("-p", "--csv_file", dest="pathcsv", default="None",
                  help="[Required] Location of the csv file containing all the data of this link: 'ftp://ftp.ncbi.nlm.nih.gov/genomes/GENOME_REPORTS/eukaryotes.txt'")

(options, args) = parser.parse_args()

pathcsv = options.pathcsv

if pathcsv == "None":
	print ("Maybe have you forget to download your data? \n -h for more information")
	sys.exit(1)

f = open(pathcsv)
alldata = [x.strip() for x in f.readlines()]
f.close()

Usage: __main__.py [options]

__main__.py: error: no such option: -f


SystemExit: 2

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [15]:
'''
Created date : 03-08-2017
checkIfInsect() function get the tax_id, species name and number acession for all species which have class == "Insects" in the csv file Eukaryotes.csv
'''

def checkIfInsect(alldata):
	insectDict = {}

	for line in alldata :
		line = line.split('\t')

		if line[5] == "Insects" :
			insectDict[line[1]] = {}
			insectDict[line[1]]["sp_name"] = line[0]
			insectDict[line[1]]["genome_id"] = line[8]

	return insectDict

_Date : 03/09/2017_

## Done
- Add addData() and createTable() function in the Python parser 
- Parser terminated
$\rightarrow$ __It works !!__

## Notes
- *__Found 247 Insect Genomes__*

In [18]:
'''
Created date : 03-09-2017
addData() function use Entrez API by BioPython to find for each tax_id : 
their family_name, family_id, order_name and order_id 
'''

def addData(insectDict) :
	for key in insectDict.keys():
		tax_id = key

		handle = Entrez.efetch(db="taxonomy", id=tax_id)
		record = Entrez.read(handle) 

		for index in range(len(record)) : 
			lineage_ex = record[index]["LineageEx"] 
			for dict in lineage_ex :

				if dict["Rank"] == "family" :
					insectDict[tax_id]["family_name"] = dict["ScientificName"]
					insectDict[tax_id]["family_id"] = dict["TaxId"]

				elif dict["Rank"] == "no rank" : 
					insectDict[tax_id]["family_name"] = "N/A"
					insectDict[tax_id]["family_id"] = "N/A"


				if dict["Rank"] == "order" : 
					insectDict[tax_id]["order_name"] = dict["ScientificName"]
					insectDict[tax_id]["order_id"] = dict["TaxId"]

				elif dict["Rank"] == "no rank" : 
					insectDict[tax_id]["order_name"] = "N/A"
					insectDict[tax_id]["order_id"] = "N/A"
					
		handle.close()

In [19]:
'''
Created date : 03-09-2017
createTable() function build the csv file which have all the information that we want 
'''

def createTable(insectDict):

	f = open("insectGenomesInfos.csv", 'w')
	f.write("Species id, Species name, Family id, Family name, Order id, Order name, Acession number\n")

	id_list = ["sp_name","family_id", "family_name", "order_id", "order_name"]

	for key in insectDict.keys():
		f.write(str(key))
		f.write(",")

		for id in range(len(id_list)) :
			value = insectDict[key][id_list[id]]

			if type(value) is int : 
				f.write(str(value))
				f.write(",")

			else :
				f.write(value)
				f.write(",")

		f.write(insectDict[key]["genome_id"])
		f.write("\n")

	f.close()